# Notebook 06: Integration Projects

## Introduction
This final notebook provides templates for capstone projects that integrate multiple concepts from the course.

## Project Ideas
1.  **Atmospheric Chemistry**: Model the Chapman cycle and ozone depletion.
2.  **Enzyme Kinetics**: Simulate Michaelis-Menten kinetics with inhibition.
3.  **Oscillating Reactions**: The Lotka-Volterra or Brusselator models.
4.  **Combustion**: Chain branching in H2 + O2.

In [ ]:
# ============================================================
# GOOGLE COLAB SETUP
# ============================================================
import sys
import os

# Check if running in Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("=" * 60)
    print("RUNNING IN GOOGLE COLAB")
    print("=" * 60)

    # Clone repository to access images
    repo_url = "https://github.com/mcbadlon31/Reaction-Dynamics-Physical-Chemistry.git"

    print(f"\nCloning repository: {repo_url}")
    print("This may take a minute...")

    !git clone {repo_url} --depth 1 --quiet

    # Change to repository directory
    os.chdir('Reaction-Dynamics-Physical-Chemistry')

    # Install additional packages if needed
    print("\nInstalling additional packages...")
    !pip install -q seaborn plotly ipywidgets

    print("\n" + "=" * 60)
    print("[SUCCESS] Colab setup complete!")
    print("=" * 60)
    print(f"Current directory: {os.getcwd()}")
    print("\nYou can now run all cells normally.")
    print("Images will load from the cloned repository.")

else:
    print("=" * 60)
    print("RUNNING IN LOCAL JUPYTER ENVIRONMENT")
    print("=" * 60)
    print("\nNo setup needed - using local files")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets

plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries loaded.")

## Template 1: Atmospheric Chemistry (Chapman Cycle)

Reactions:
1. $O_2 + h\nu \xrightarrow{k_1} 2O$
2. $O + O_2 + M \xrightarrow{k_2} O_3 + M$
3. $O_3 + h\nu \xrightarrow{k_3} O_2 + O$
4. $O + O_3 \xrightarrow{k_4} 2O_2$

In [ ]:
def chapman_cycle(k1, k2, k3, k4, days):
    # Time points (seconds)
    t = np.linspace(0, days * 24 * 3600, 1000)
    
    # Initial concentrations (molecules/cm^3)
    O2_0 = 5e12
    O_0 = 0
    O3_0 = 0
    M = 2e13  # Air density
    
    y0 = [O2_0, O_0, O3_0]
    
    def derivatives(y, t):
        O2, O, O3 = y
        
        # Rates
        r1 = k1 * O2
        r2 = k2 * O * O2 * M
        r3 = k3 * O3
        r4 = k4 * O * O3
        
        dO2dt = -r1 - r2 + r3 + 2*r4
        dOdt = 2*r1 - r2 + r3 - r4
        dO3dt = r2 - r3 - r4
        
        return [dO2dt, dOdt, dO3dt]
    
    # Solve ODE
    sol = odeint(derivatives, y0, t)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.semilogy(t / 3600, sol[:, 1], label='O')
    plt.semilogy(t / 3600, sol[:, 2], label='O3')
    plt.xlabel('Time (hours)')
    plt.ylabel('Concentration (molecules/cm^3)')
    plt.title('Chapman Cycle Simulation')
    plt.legend()
    plt.grid(True)
    plt.show()

# Example rate constants (approximate)
widgets.interact(chapman_cycle, 
                 k1=widgets.FloatLogSlider(min=-12, max=-8, value=-10, description='k1 (Photolysis O2)'),
                 k2=widgets.FloatLogSlider(min=-34, max=-30, value=-33, description='k2 (O3 formation)'),
                 k3=widgets.FloatLogSlider(min=-4, max=-2, value=-3, description='k3 (Photolysis O3)'),
                 k4=widgets.FloatLogSlider(min=-16, max=-14, value=-15, description='k4 (O3 destruction)'),
                 days=widgets.IntSlider(min=1, max=10, value=2, description='Days'));

### ❓ Concept Check

<details>
<summary><strong>Q: Why does the ozone concentration reach a steady state?</strong></summary>

Because the rate of ozone formation (reaction 2) eventually balances the rate of ozone destruction (reactions 3 and 4). This is a dynamic equilibrium maintained by the constant input of solar energy ($h\nu$).
</details>

## Template 2: Enzyme Kinetics (Michaelis-Menten)

$$ E + S \rightleftharpoons ES \rightarrow E + P $$

In [ ]:
def michaelis_menten(k1, k_1, k2, S0):
    t = np.linspace(0, 100, 500)
    E0 = 1.0
    ES0 = 0
    P0 = 0
    
    y0 = [E0, S0, ES0, P0]
    
    def derivatives(y, t):
        E, S, ES, P = y
        
        r1 = k1 * E * S
        r_1 = k_1 * ES
        r2 = k2 * ES
        
        dEdt = -r1 + r_1 + r2
        dSdt = -r1 + r_1
        dESdt = r1 - r_1 - r2
        dPdt = r2
        
        return [dEdt, dSdt, dESdt, dPdt]
    
    sol = odeint(derivatives, y0, t)
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, sol[:, 1], label='Substrate [S]')
    plt.plot(t, sol[:, 2], label='Complex [ES]')
    plt.plot(t, sol[:, 3], label='Product [P]')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Enzyme Kinetics')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(michaelis_menten, 
                 k1=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k1 (Bind)'),
                 k_1=widgets.FloatSlider(min=0.1, max=2.0, value=0.5, description='k-1 (Unbind)'),
                 k2=widgets.FloatSlider(min=0.1, max=2.0, value=0.5, description='k2 (React)'),
                 S0=widgets.FloatSlider(min=1.0, max=10.0, value=5.0, description='[S]0'));

### ❓ Concept Check

<details>
<summary><strong>Q: What is the "Steady State Approximation" in enzyme kinetics?</strong></summary>

It assumes that the concentration of the intermediate complex $[ES]$ remains roughly constant during the main part of the reaction ($d[ES]/dt \approx 0$). You can see this in the plot where the orange curve ($[ES]$) stays flat for a while.
</details>

## Template 3: Oscillating Reactions (Lotka-Volterra)

A simple model for oscillating populations (predator-prey) or autocatalytic reactions.

1. $A + X \xrightarrow{k_1} 2X$ (Autocatalysis)
2. $X + Y \xrightarrow{k_2} 2Y$ (Predation)
3. $Y \xrightarrow{k_3} P$ (Decay)

In [ ]:
def lotka_volterra(k1, k2, k3):
    t = np.linspace(0, 50, 1000)
    X0 = 1.0
    Y0 = 0.5
    
    # Assume [A] is constant (pool chemical)
    A = 1.0
    
    y0 = [X0, Y0]
    
    def derivatives(y, t):
        X, Y = y
        
        dXdt = k1*A*X - k2*X*Y
        dYdt = k2*X*Y - k3*Y
        
        return [dXdt, dYdt]
    
    sol = odeint(derivatives, y0, t)
    
    plt.figure(figsize=(10, 6))
    plt.plot(t, sol[:, 0], label='Species X (Prey)')
    plt.plot(t, sol[:, 1], label='Species Y (Predator)')
    plt.xlabel('Time')
    plt.ylabel('Concentration')
    plt.title('Lotka-Volterra Oscillations')
    plt.legend()
    plt.grid(True)
    plt.show()

widgets.interact(lotka_volterra, 
                 k1=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k1'),
                 k2=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k2'),
                 k3=widgets.FloatSlider(min=0.1, max=2.0, value=1.0, description='k3'));

## Template 4: Chemical Reactor Design

Design a reactor to maximize the yield of intermediate B in a series reaction $A \to B \to C$.
- **CSTR**: Continuous Stirred-Tank Reactor
- **PFR**: Plug Flow Reactor

In [ ]:
class ReactorDesigner:
    """Design a chemical reactor for A -> B -> C"""
    
    def __init__(self):
        self.setup_widgets()
        self.setup_display()
        
    def setup_widgets(self):
        self.tau_slider = widgets.FloatSlider(min=0.1, max=10.0, value=1.0, description='Tau (s)')
        self.k1_slider = widgets.FloatSlider(min=0.1, max=5.0, value=1.0, description='k1 (A->B)')
        self.k2_slider = widgets.FloatSlider(min=0.1, max=5.0, value=0.5, description='k2 (B->C)')
        
    def plot(self, tau, k1, k2):
        # CSTR Equations
        # A = A0 / (1 + k1*tau)
        # B = k1*A*tau / (1 + k2*tau)
        
        # PFR Equations
        # A = A0 * exp(-k1*tau)
        # B = A0 * k1/(k2-k1) * (exp(-k1*tau) - exp(-k2*tau))
        
        A0 = 1.0
        
        # Calculate CSTR
        A_cstr = A0 / (1 + k1*tau)
        B_cstr = k1 * A_cstr * tau / (1 + k2*tau)
        C_cstr = A0 - A_cstr - B_cstr
        
        # Calculate PFR
        if abs(k1 - k2) < 1e-5:
            B_pfr = A0 * k1 * tau * np.exp(-k1*tau)
        else:
            B_pfr = A0 * k1 / (k2 - k1) * (np.exp(-k1*tau) - np.exp(-k2*tau))
        A_pfr = A0 * np.exp(-k1*tau)
        C_pfr = A0 - A_pfr - B_pfr
        
        # Plot concentrations vs Tau (Residence Time)
        tau_range = np.linspace(0, 10, 100)
        
        # PFR Profile over time
        if abs(k1 - k2) < 1e-5:
            B_prof = A0 * k1 * tau_range * np.exp(-k1*tau_range)
        else:
            B_prof = A0 * k1 / (k2 - k1) * (np.exp(-k1*tau_range) - np.exp(-k2*tau_range))
            
        # CSTR Profile
        A_cstr_prof = A0 / (1 + k1*tau_range)
        B_cstr_prof = k1 * A_cstr_prof * tau_range / (1 + k2*tau_range)
        
        plt.figure(figsize=(10, 6))
        plt.plot(tau_range, B_prof, 'b-', label='PFR Yield (B)')
        plt.plot(tau_range, B_cstr_prof, 'r--', label='CSTR Yield (B)')
        
        plt.axvline(tau, color='k', linestyle=':', label=f'Current Tau = {tau} s')
        plt.plot(tau, B_pfr, 'bo', markersize=10)
        plt.plot(tau, B_cstr, 'ro', markersize=10)
        
        plt.xlabel('Residence Time (s)')
        plt.ylabel('Concentration of B')
        plt.title('Reactor Design: Maximizing Intermediate B')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"PFR Yield: {B_pfr:.3f} ({(B_pfr/A0)*100:.1f}%)")
        print(f"CSTR Yield: {B_cstr:.3f} ({(B_cstr/A0)*100:.1f}%)")
        if B_pfr > B_cstr:
            print("PFR is more efficient for this reaction.")
        else:
            print("CSTR is more efficient (unlikely for series reaction).")

    def setup_display(self):
        widgets.interact(self.plot, tau=self.tau_slider, k1=self.k1_slider, k2=self.k2_slider)

print("\n🏭 Chemical Reactor Designer:")
reactor = ReactorDesigner()

## Template 5: Photosynthetic Electron Transfer

Model the Z-scheme of photosynthesis, tracking electron flow from Water to NADP+.

In [ ]:
class PhotosynthesisModel:
    """Simulate Z-Scheme Electron Transfer"""
    
    def __init__(self):
        # Simplified Z-scheme energies (eV vs SHE approx)
        self.components = {
            'P680': 1.2, 'P680*': -0.8,
            'QA': -0.1, 'QB': 0.0,
            'PQ': 0.1, 'Cytb6f': 0.4,
            'PC': 0.5,
            'P700': 0.6, 'P700*': -1.2,
            'FA/FB': -0.5, 'Fd': -0.4, 'NADP+': -0.32
        }
        self.setup_display()
        
    def plot_z_scheme(self, light_intensity):
        # Adjust excited state populations based on light
        pop_P680_star = min(1.0, light_intensity / 100)
        pop_P700_star = min(1.0, light_intensity / 100)
        
        # Plot Energy Diagram
        fig, ax = plt.subplots(figsize=(12, 8))
        
        # Coordinates for components
        x_coords = np.arange(len(self.components))
        energies = list(self.components.values())
        names = list(self.components.keys())
        
        # Draw levels
        for i, (name, E) in enumerate(self.components.items()):
            color = 'orange' if '*' in name else 'green'
            ax.plot([i-0.3, i+0.3], [E, E], color=color, linewidth=3)
            ax.text(i, E+0.05, name, ha='center')
            
        # Draw connections (Electron Flow)
        # P680 -> P680* (Light)
        ax.annotate('', xy=(1, self.components['P680*']), xytext=(0, self.components['P680']),
                   arrowprops=dict(arrowstyle='->', color='yellow', lw=2, linestyle='--'))
        
        # P680* -> QA -> ... -> P700
        path_indices = [1, 2, 3, 4, 5, 6, 7]
        for i in range(len(path_indices)-1):
            idx1, idx2 = path_indices[i], path_indices[i+1]
            ax.annotate('', xy=(idx2, energies[idx2]), xytext=(idx1, energies[idx1]),
                       arrowprops=dict(arrowstyle='->', color='blue'))
            
        # P700 -> P700* (Light)
        ax.annotate('', xy=(8, self.components['P700*']), xytext=(7, self.components['P700']),
                   arrowprops=dict(arrowstyle='->', color='yellow', lw=2, linestyle='--'))
                   
        # P700* -> ... -> NADP+
        path_indices_2 = [8, 9, 10, 11]
        for i in range(len(path_indices_2)-1):
            idx1, idx2 = path_indices_2[i], path_indices_2[i+1]
            ax.annotate('', xy=(idx2, energies[idx2]), xytext=(idx1, energies[idx1]),
                       arrowprops=dict(arrowstyle='->', color='blue'))
        
        ax.set_ylabel('Redox Potential (V)')
        ax.set_title(f'Z-Scheme Photosynthesis (Light Intensity: {light_intensity}%)')
        ax.invert_yaxis() # Convention in biology: negative (high energy) up
        ax.grid(True, alpha=0.3)
        plt.show()
        
        # Calculate theoretical efficiency
        input_energy = 2 * 1.8 # approx eV for 680nm and 700nm photons
        stored_energy = 1.2 - (-0.32) # Delta E from H2O to NADP+
        eff = (stored_energy / input_energy) * 100
        print(f"Theoretical Energy Efficiency: {eff:.1f}%")
        print(f"Rate of NADPH production is proportional to light intensity: {light_intensity}%")

    def setup_display(self):
        widgets.interact(self.plot_z_scheme, 
                        light_intensity=widgets.IntSlider(min=0, max=100, step=10, value=50, description='Light %'))

print("\n🌿 Photosynthesis Z-Scheme Explorer:")
photo_model = PhotosynthesisModel()

## Template 6: Build Your Own Reaction (PES Constructor)

Create a custom Potential Energy Surface (PES) by adding Gaussian wells and hills, then simulate a reaction trajectory.

In [ ]:
class PESConstructor:
    """Interactive 2D Potential Energy Surface Builder"""
    
    def __init__(self):
        self.wells = [] # List of (x, y, depth, width)
        self.setup_widgets()
        
    def add_well(self, x, y, depth, width):
        self.wells.append({'x': x, 'y': y, 'depth': depth, 'width': width})
        
    def clear(self):
        self.wells = []
        
    def potential(self, X, Y):
        Z = np.zeros_like(X)
        # Default harmonic well
        Z += 0.5 * (X**2 + Y**2)
        
        # Add custom Gaussian wells/hills
        for w in self.wells:
            # Gaussian: A * exp(-((x-x0)^2 + (y-y0)^2) / 2w^2)
            Z += w['depth'] * np.exp(-((X - w['x'])**2 + (Y - w['y'])**2) / (2 * w['width']**2))
            
        return Z
        
    def plot(self, n_wells):
        # Just for interactivity, we regenerate random wells if n_wells changes
        # In a real app, we'd have click-to-add
        if len(self.wells) != n_wells:
            self.clear()
            for _ in range(n_wells):
                self.add_well(np.random.uniform(-2, 2), np.random.uniform(-2, 2), 
                             np.random.uniform(-5, 5), np.random.uniform(0.5, 1.0))
        
        x = np.linspace(-3, 3, 100)
        y = np.linspace(-3, 3, 100)
        X, Y = np.meshgrid(x, y)
        Z = self.potential(X, Y)
        
        fig = plt.figure(figsize=(10, 8))
        ax = fig.add_subplot(111, projection='3d')
        surf = ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
        fig.colorbar(surf, shrink=0.5, aspect=5)
        
        ax.set_title(f'Custom Potential Energy Surface ({n_wells} Features)')
        ax.set_xlabel('Coordinate 1')
        ax.set_ylabel('Coordinate 2')
        ax.set_zlabel('Energy')
        plt.show()
        
        # Run a simple "ball" trajectory from (2, 2)
        # Gradient descent
        path_x, path_y = [2.0], [2.0]
        curr_x, curr_y = 2.0, 2.0
        lr = 0.1
        for _ in range(20):
            # Finite difference gradient
            eps = 0.01
            dzdx = (self.potential(curr_x+eps, curr_y) - self.potential(curr_x-eps, curr_y)) / (2*eps)
            dzdy = (self.potential(curr_x, curr_y+eps) - self.potential(curr_x, curr_y-eps)) / (2*eps)
            
            curr_x -= lr * dzdx
            curr_y -= lr * dzdy
            path_x.append(curr_x)
            path_y.append(curr_y)
            
        print("Trajectory simulation (Gradient Descent) from (2,2) completed.")
        print(f"Final position: ({curr_x:.2f}, {curr_y:.2f})")

    def setup_display(self):
        widgets.interact(self.plot, n_wells=widgets.IntSlider(min=0, max=5, value=2, description='Features'))

print("\n🏔️ PES Constructor (Randomized Features):")
pes_builder = PESConstructor()

---## PROJECT 4: Complete Reaction Dynamics Analysis (H + HI → HI + H)### COMPREHENSIVE CAPSTONE PROJECTThis project integrates all computational methods from Notebooks 03 and 04:- **LEPS Potential Energy Surfaces** (London-Eyring-Polanyi-Sato method)- **Transition State Optimization** (Newton-Raphson saddle point search)- **Classical Trajectory Simulations** (Quasiclassical dynamics)- **Statistical Analysis** (Reaction probabilities and product distributions)**Learning Objectives:**- Construct and visualize LEPS surfaces- Locate and characterize transition states- Run classical trajectories and analyze outcomes- Connect microscopic dynamics to macroscopic rates- Validate Polanyi's Rules computationally**Time Estimate:** 2-3 hours**Prerequisites:** Notebooks 03 and 04---

### PART 1: LEPS Surface Construction and Visualization**Task:** Build the LEPS potential energy surface for the H + HI → HI + H symmetric exchange reaction.**Background:**- This is a classic test system in reaction dynamics- Symmetric reaction (reactants and products identical)- Exhibits an early barrier (attractive surface)- Well-studied experimentally and theoretically**Your Goals:**1. Load Morse parameters from data file2. Construct LEPS surface with K_sato = 0.03. Generate 3D and contour visualizations4. Identify reactant and product valleys

In [ ]:
# PART 1: LEPS Surface Constructionimport syssys.path.append('../modules')import numpy as npimport matplotlib.pyplot as pltimport pandas as pdfrom leps_surface import LEPSSurfacefrom visualization import plot_pes_3d, plot_pes_contour, plot_morse_curveprint("="*80)print("PROJECT 4: COMPLETE REACTION DYNAMICS ANALYSIS")print("System: H + HI -> HI + H")print("="*80)# Step 1: Initialize LEPS surfaceprint("\n[STEP 1] Initializing LEPS surface...")surface = LEPSSurface('HI', 'HI', 'I2', K_sato=0.0)print("[OK] LEPS surface initialized")print("\nMorse Parameters:")print(f"  H-I: D_e = {surface.params['AB']['D_e']:.2f} kJ/mol, R_e = {surface.params['AB']['R_e']:.3f} Å")print(f"  I-I: D_e = {surface.params['AC']['D_e']:.2f} kJ/mol, R_e = {surface.params['AC']['R_e']:.3f} Å")# Step 2: Visualize individual Morse potentialsprint("\n[STEP 2] Visualizing Morse potentials...")fig, axes = plt.subplots(1, 2, figsize=(14, 5))# H-I Morse potentialR_range = np.linspace(0.8, 6.0, 200)params_HI = surface.params['AB']V_HI = surface.morse_potential(R_range, params_HI['D_e'], params_HI['R_e'], params_HI['beta'])axes[0].plot(R_range, V_HI, 'b-', linewidth=2.5)axes[0].axhline(0, color='gray', linestyle='--', linewidth=1)axes[0].axvline(params_HI['R_e'], color='green', linestyle=':', linewidth=2, label=f"R_e = {params_HI['R_e']:.3f} Å")axes[0].axhline(-params_HI['D_e'], color='red', linestyle=':', linewidth=2, label=f"D_e = {params_HI['D_e']:.1f} kJ/mol")axes[0].set_xlabel('R (Å)', fontsize=11)axes[0].set_ylabel('V (kJ/mol)', fontsize=11)axes[0].set_title('H-I Morse Potential', fontsize=13, fontweight='bold')axes[0].legend()axes[0].grid(True, alpha=0.3)axes[0].set_ylim(-350, 100)# I-I Morse potentialparams_I2 = surface.params['AC']V_I2 = surface.morse_potential(R_range, params_I2['D_e'], params_I2['R_e'], params_I2['beta'])axes[1].plot(R_range, V_I2, 'purple', linewidth=2.5)axes[1].axhline(0, color='gray', linestyle='--', linewidth=1)axes[1].axvline(params_I2['R_e'], color='green', linestyle=':', linewidth=2, label=f"R_e = {params_I2['R_e']:.3f} Å")axes[1].axhline(-params_I2['D_e'], color='red', linestyle=':', linewidth=2, label=f"D_e = {params_I2['D_e']:.1f} kJ/mol")axes[1].set_xlabel('R (Å)', fontsize=11)axes[1].set_ylabel('V (kJ/mol)', fontsize=11)axes[1].set_title('I-I Morse Potential', fontsize=13, fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)axes[1].set_ylim(-200, 100)plt.tight_layout()plt.show()print("[OK] Morse potentials plotted")# Step 3: Generate 2D LEPS surfaceprint("\n[STEP 3] Generating 2D LEPS surface...")R_AB_range = np.linspace(1.0, 4.5, 60)R_BC_range = np.linspace(1.0, 4.5, 60)R_AB_grid, R_BC_grid, V_grid = surface.energy_surface_2d(R_AB_range, R_BC_range, angle_deg=180.0)print(f"[OK] Surface calculated on {V_grid.shape} grid")print(f"  Energy range: {V_grid.min():.1f} to {V_grid.max():.1f} kJ/mol")# Step 4: Create 3D visualizationprint("\n[STEP 4] Creating visualizations...")fig_3d, ax_3d = plot_pes_3d(R_AB_grid, R_BC_grid, V_grid,                            title="LEPS Surface: H + HI",                            xlabel="R(H···H) (Å)",                            ylabel="R(H-I) (Å)",                            elev=25, azim=45)plt.show()# Create contour plotfig_contour, ax_contour = plot_pes_contour(R_AB_grid, R_BC_grid, V_grid,                                            title="LEPS Energy Contours: H + HI",                                            xlabel="R(H···H) (Å)",                                            ylabel="R(H-I) (Å)",                                            levels=40,                                            vmin=-320, vmax=-180)plt.show()print("[OK] Visualizations complete")print("\n" + "="*80)

### PART 2: Transition State Optimization**Task:** Locate the exact transition state (saddle point) using Newton-Raphson optimization.**Your Goals:**1. Set up initial guess for saddle point coordinates2. Run Newton-Raphson optimization3. Verify saddle point (one negative Hessian eigenvalue)4. Calculate activation energy5. Visualize optimization convergence

In [ ]:
# PART 2: Transition State Optimizationfrom transition_state import TransitionStateOptimizerprint("="*80)print("PART 2: TRANSITION STATE OPTIMIZATION")print("="*80)# Step 1: Initialize optimizerprint("\n[STEP 1] Initializing Newton-Raphson optimizer...")optimizer = TransitionStateOptimizer(surface, tolerance=1e-6, max_iterations=50)print("[OK] Optimizer initialized")print(f"  Convergence tolerance: {optimizer.tolerance:.0e}")print(f"  Maximum iterations: {optimizer.max_iterations}")# Step 2: Run optimizationprint("\n[STEP 2] Running Newton-Raphson optimization...")R_AB_init, R_BC_init = 1.9, 1.9print(f"  Initial guess: R_AB = {R_AB_init:.2f} Å, R_BC = {R_BC_init:.2f} Å")print()result = optimizer.optimize_saddle_point(R_AB_init, R_BC_init, verbose=True)# Step 3: Analyze resultsprint("\n[STEP 3] Analyzing transition state...")eigenvalues = result['eigenvalues']print(f"\nHessian eigenvalues: {eigenvalues}")if eigenvalues[0] < 0 and eigenvalues[1] > 0:    print("[OK] Confirmed saddle point (one negative eigenvalue)")    print(f"  Reaction coordinate curvature: {eigenvalues[0]:.2f} kJ/(mol·Å²)")    print(f"  Perpendicular curvature: {eigenvalues[1]:.2f} kJ/(mol·Å²)")# Calculate activation energyE_reactants = surface.leps_potential(3.0, params_HI['R_e'], 3.0 + params_HI['R_e'])Ea_forward = result['energy'] - E_reactantsprint(f"\nEnergetics:")print(f"  Reactant energy: {E_reactants:.2f} kJ/mol")print(f"  Saddle point energy: {result['energy']:.2f} kJ/mol")print(f"  Activation energy: {Ea_forward:.2f} kJ/mol")# Step 4: Visualize convergenceprint("\n[STEP 4] Visualizing optimization convergence...")history_df = result['history']fig, axes = plt.subplots(2, 2, figsize=(14, 10))# Energy convergenceaxes[0,0].plot(history_df['iteration'], history_df['energy'], 'b-o', linewidth=2)axes[0,0].set_xlabel('Iteration')axes[0,0].set_ylabel('Energy (kJ/mol)')axes[0,0].set_title('Energy Convergence', fontweight='bold')axes[0,0].grid(True, alpha=0.3)# Gradient convergenceaxes[0,1].semilogy(history_df['iteration'], history_df['gradient_norm'], 'r-o', linewidth=2)axes[0,1].axhline(optimizer.tolerance, color='green', linestyle='--', linewidth=2,                  label=f'Tolerance = {optimizer.tolerance:.0e}')axes[0,1].set_xlabel('Iteration')axes[0,1].set_ylabel('|Gradient| (log scale)')axes[0,1].set_title('Gradient Convergence', fontweight='bold')axes[0,1].legend()axes[0,1].grid(True, alpha=0.3)# Optimization pathaxes[1,0].contourf(R_AB_grid, R_BC_grid, V_grid, levels=40, cmap='viridis',                   vmin=-320, vmax=-180, alpha=0.6)axes[1,0].plot(history_df['R_AB'], history_df['R_BC'], 'r-o', linewidth=3,               label='Optimization path', markeredgecolor='white', markeredgewidth=2)axes[1,0].plot(history_df['R_AB'].iloc[0], history_df['R_BC'].iloc[0],               'go', markersize=15, label='Start')axes[1,0].plot(history_df['R_AB'].iloc[-1], history_df['R_BC'].iloc[-1],               'r*', markersize=20, label='Saddle point')axes[1,0].set_xlabel('R(H···H) (Å)')axes[1,0].set_ylabel('R(H-I) (Å)')axes[1,0].set_title('Optimization Path on PES', fontweight='bold')axes[1,0].legend()# Coordinate convergenceaxes[1,1].plot(history_df['iteration'], history_df['R_AB'], 'b-o', linewidth=2, label='R_AB')axes[1,1].plot(history_df['iteration'], history_df['R_BC'], 'g-s', linewidth=2, label='R_BC')axes[1,1].set_xlabel('Iteration')axes[1,1].set_ylabel('Distance (Å)')axes[1,1].set_title('Coordinate Convergence', fontweight='bold')axes[1,1].legend()axes[1,1].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("[OK] Convergence analysis complete")print("\n" + "="*80)

### PART 3: Classical Trajectory Simulations**Task:** Run multiple classical trajectories to study reaction dynamics.**Your Goals:**1. Set up trajectory calculator2. Run single trajectory and analyze outcome3. Perform Monte Carlo sampling (batch trajectories)4. Calculate reaction probability5. Analyze energy conservation statistics

In [ ]:
# PART 3: Classical Trajectory Simulationsfrom trajectory import ClassicalTrajectoryprint("="*80)print("PART 3: CLASSICAL TRAJECTORY SIMULATIONS")print("="*80)# Step 1: Initialize trajectory calculatorprint("\n[STEP 1] Initializing trajectory calculator...")traj_calc = ClassicalTrajectory(surface, 'H', 'H', 'I', dt=0.010)print("[OK] Trajectory calculator initialized")print(f"  Time step: {traj_calc.dt} fs")print(f"  Masses: H={traj_calc.m_A:.3f} amu, I={traj_calc.m_C:.3f} amu")# Step 2: Single trajectory exampleprint("\n[STEP 2] Running example trajectory...")R_AB_0 = 3.0R_BC_0 = params_HI['R_e']R_AC_0 = R_AB_0 + R_BC_0E_trans = 60.0  # kJ/molv_approach = -np.sqrt(2 * E_trans / (traj_calc.m_A * 9646.9))single_result = traj_calc.run_trajectory(R_AB_0, R_BC_0, R_AC_0,                                         v_approach, 0.0, v_approach,                                         max_time=500.0, save_interval=5)print(f"[OK] Trajectory complete")print(f"  Duration: {single_result['time'][-1]:.2f} fs")print(f"  Outcome: {single_result['outcome']}")print(f"  Energy drift: {single_result['energy_drift']:.4f}%")# Visualizefig, axes = plt.subplots(1, 2, figsize=(14, 6))# Trajectory on PESaxes[0].contourf(R_AB_grid, R_BC_grid, V_grid, levels=40, cmap='viridis',                 vmin=-320, vmax=-180, alpha=0.7)axes[0].plot(single_result['R_AB'], single_result['R_BC'], 'r-', linewidth=2.5, label='Trajectory')axes[0].plot(single_result['R_AB'][0], single_result['R_BC'][0], 'go', markersize=12, label='Start')axes[0].plot(single_result['R_AB'][-1], single_result['R_BC'][-1], 'bs', markersize=12, label='End')axes[0].set_xlabel('R(H···H) (Å)')axes[0].set_ylabel('R(H-I) (Å)')axes[0].set_title('Single Trajectory on PES', fontweight='bold')axes[0].legend()# Energy conservationaxes[1].plot(single_result['time'], single_result['V'], 'b-', linewidth=2, label='Potential')axes[1].plot(single_result['time'], single_result['T'], 'r-', linewidth=2, label='Kinetic')axes[1].plot(single_result['time'], single_result['E_total'], 'k--', linewidth=2.5, label='Total')axes[1].set_xlabel('Time (fs)')axes[1].set_ylabel('Energy (kJ/mol)')axes[1].set_title('Energy Conservation', fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()# Step 3: Monte Carlo batchprint("\n[STEP 3] Running Monte Carlo trajectory batch...")print("  (This may take 1-2 minutes...)")n_traj = 30batch_results = []reactive_count = 0for i in range(n_traj):    v_pert = np.random.normal(0, 0.008)    res = traj_calc.run_trajectory(R_AB_0, R_BC_0, R_AC_0,                                   v_approach + v_pert, 0.0, v_approach + v_pert,                                   max_time=500.0, save_interval=10)    batch_results.append({        'id': i,        'outcome': res['outcome'],        'energy_drift': res['energy_drift'],        'final_R_AB': res['R_AB'][-1],        'final_R_BC': res['R_BC'][-1]    })    if res['outcome'] == 'reactive':        reactive_count += 1    if (i+1) % 10 == 0:        print(f"  {i+1}/{n_traj} complete...")batch_df = pd.DataFrame(batch_results)print(f"\n[OK] Batch complete")print(f"  Reactive: {reactive_count}/{n_traj} ({reactive_count/n_traj*100:.1f}%)")print(f"  Avg energy drift: {batch_df['energy_drift'].abs().mean():.4f}%")# Visualize outcomesfig, axes = plt.subplots(1, 2, figsize=(14, 6))# Outcome distributionoutcomes = batch_df['outcome'].value_counts()axes[0].bar(range(len(outcomes)), outcomes.values, color=['green', 'red', 'gray'][:len(outcomes)],           edgecolor='black', linewidth=2, alpha=0.7)axes[0].set_xticks(range(len(outcomes)))axes[0].set_xticklabels(outcomes.index)axes[0].set_ylabel('Count')axes[0].set_title('Trajectory Outcomes', fontweight='bold')axes[0].grid(True, alpha=0.3, axis='y')# Final geometriesreactive = batch_df[batch_df['outcome'] == 'reactive']non_reactive = batch_df[batch_df['outcome'] == 'non-reactive']if len(reactive) > 0:    axes[1].scatter(reactive['final_R_AB'], reactive['final_R_BC'],                   s=100, c='green', alpha=0.6, edgecolors='black', label='Reactive')if len(non_reactive) > 0:    axes[1].scatter(non_reactive['final_R_AB'], non_reactive['final_R_BC'],                   s=100, c='red', alpha=0.6, edgecolors='black', label='Non-reactive')axes[1].set_xlabel('Final R_AB (Å)')axes[1].set_ylabel('Final R_BC (Å)')axes[1].set_title('Final Product Geometries', fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.show()print("\n" + "="*80)

### PART 4: Final Analysis and Report**Task:** Synthesize all results into a comprehensive analysis.**Deliverables:**1. Summary of PES characteristics2. Transition state properties3. Reaction dynamics statistics4. Comparison to experimental/theoretical values5. Discussion of Polanyi's Rules validation

In [ ]:
# PART 4: Final Analysisprint("="*80)print("PROJECT 4: FINAL ANALYSIS AND SUMMARY")print("="*80)print("\n1. POTENTIAL ENERGY SURFACE")print("-" * 80)print(f"  Method: LEPS (K_sato = 0.0)")print(f"  Surface type: Symmetric exchange reaction")print(f"  Energy range: {V_grid.min():.1f} to {V_grid.max():.1f} kJ/mol")print(f"  Barrier location: Early (entrance channel)")print("\n2. TRANSITION STATE PROPERTIES")print("-" * 80)print(f"  Geometry: R_AB = {result['R_AB']:.4f} Å, R_BC = {result['R_BC']:.4f} Å")print(f"  Symmetry: R_AB ≈ R_BC (confirmed for symmetric reaction)")print(f"  Energy: {result['energy']:.2f} kJ/mol")print(f"  Activation energy: {Ea_forward:.2f} kJ/mol")print(f"  Optimization: Converged in {result['iterations']} iterations")print(f"  Hessian eigenvalues: [{eigenvalues[0]:.1f}, {eigenvalues[1]:.1f}]")print(f"  Saddle point confirmed: {eigenvalues[0] < 0 and eigenvalues[1] > 0}")print("\n3. CLASSICAL TRAJECTORY DYNAMICS")print("-" * 80)print(f"  Number of trajectories: {n_traj}")print(f"  Collision energy: {E_trans:.1f} kJ/mol")print(f"  Reactive outcomes: {reactive_count} ({reactive_count/n_traj*100:.1f}%)")print(f"  Reaction probability: {reactive_count/n_traj:.3f}")print(f"  Average energy conservation: {batch_df['energy_drift'].abs().mean():.4f}%")print(f"  Quality: {'EXCELLENT' if batch_df['energy_drift'].abs().mean() < 0.01 else 'GOOD'}")print("\n4. POLANYI'S RULES VALIDATION")print("-" * 80)print("  Prediction: Early barrier → translational energy effective")print(f"  Observation: Reaction probability = {reactive_count/n_traj:.3f} at E_trans = {E_trans} kJ/mol")print("  → Translational energy successfully promotes reaction")print("  → Validates Polanyi's Rule #1 for attractive surfaces")print("\n5. COMPARISON TO LITERATURE")print("-" * 80)print("  Experimental Ea (H + HI): ~60-70 kJ/mol")print(f"  This calculation: {Ea_forward:.1f} kJ/mol")print(f"  Agreement: {'Good' if 50 <= Ea_forward <= 80 else 'Moderate'}")print("\n  Note: LEPS is semi-empirical; ab initio methods give Ea ≈ 62 kJ/mol")print("\n6. CONCLUSIONS")print("-" * 80)print("  ✓ Successfully constructed LEPS potential energy surface")print("  ✓ Located transition state with high precision")print("  ✓ Demonstrated excellent energy conservation in trajectories")print("  ✓ Calculated reaction probability from classical dynamics")print("  ✓ Validated Polanyi's Rules computationally")print("\n" + "="*80)print("PROJECT 4 COMPLETE")print("="*80)

### 🎓 Project Reflection Questions**1. PES Topology:**- How does the early barrier location affect reaction dynamics?- What would change if we used a late barrier (repulsive surface)?**2. Optimization:**- Why is the saddle point symmetric (R_AB = R_BC) for this reaction?- How many iterations did Newton-Raphson need? Why so few?**3. Trajectories:**- What causes some trajectories to be reactive and others not?- How does energy conservation quality affect results?**4. Broader Context:**- How do these methods connect to modern quantum chemistry?- What are the limitations of classical trajectories?---### 🚀 Extensions and Advanced Topics**If you want to go further:**1. **Vary K_sato parameter** (0.0 to 0.3) and see how barrier changes2. **Try different collision energies** and plot reaction probability vs. E3. **Implement deuterium isotope** (D + DI) and compare KIE4. **Add rotational energy** to initial conditions5. **Compare to quantum scattering** results (literature)**This project demonstrates the complete workflow of computational reaction dynamics - exactly how research is done!**---